In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import xgboost as xgb
%matplotlib inline

In [2]:
if 'ygong' in os.getcwd():
    filepath = "../data"
    dir_out = "../output"
else:
    filepath = "/home/gong/Documents/Kaggle_July2019/data"
    dir_out = "/home/gong/Documents/Kaggle_July2019/output"

In [23]:
def prepare_train():
    train = pd.read_csv(os.path.join(dir_out, '_train.csv'))
    test = pd.read_csv(os.path.join(dir_out, '_test.csv'))
    #Get only 10% of dataset for fast evaluation!
    size = round(0.10 * train.shape[0])
    train = train[:size]
    print('Train dataset shape is now rows: {} cols:{}'.format(train.shape[0], train.shape[1]))

    train_drop_list = ['scalar_coupling_constant', 'atom_index_1', 'atom_index_0', 'id', 'type', 'type_bond',
                        'atom_0', 'atom_1', 'molecule_name']
    test_drop_list = list(set(train_drop_list) - set(['scalar_coupling_constant']))
    
    X_train = train.drop(columns=train_drop_list)
    X_test = test.drop(columns=test_drop_list)
    y = train['scalar_coupling_constant'].values

    return X_train, X_test, y

In [22]:
X_train, X_test, y_train = prepare_train()
# initialize XGBRegressor
xgb_model = xgb.XGBRegressor(n_estimators=10000, seed=42, n_jobs=8, objective="reg:squarederror")

xgb_model.fit(X_train, y_train)
y_predict_xgb = xgb_model.predict(X_test)
print('Train score: {}'.format(xgb_model.score(X_train, y_train)))

submit = pd.read_csv(os.path.join(filepath, 'sample_submission.csv')).drop(columns='scalar_coupling_constant')
submit['scalar_coupling_constant'] = y_predict_xgb
submit.to_csv(os.path.join(dir_out, 'submission.csv'), index=False)

fig, axes = plt.subplots()
fig.set_size_inches(16, 12)
xgb.plot_importance(xgb_model, ax=axes)
plt.show()

Train dataset shape is now rows: 465815 cols:18


ValueError: feature_names mismatch: ['n_bonds', 'cov', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1', 'dist'] ['n_bonds', 'cov', 'dist']
expected y_0, y_1, x_1, z_0, x_0, z_1 in input data

In [28]:
import subprocess
#subprocess.check_output(['ls','-l']) #all that is technically needed...
print(subprocess.check_output(['pwd']))
subprocess.call(['sh', './submit.sh'])

b'/home/gong/Documents/Kaggle_July2019/code\n'


0